#CS 410 Final Project: Visualizing Reddit Location Data

[insert description]

##Part 1: Set-Up

In [ ]:
!pip install contractions
!pip install rank_bm25
!pip install transformers==3.1.0

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 287 kB 14.4 MB/s 
     |████████████████████████████████| 110 kB 23.6 MB/s 
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 884 kB 12.3 MB/s 
     |████████████████████████████████| 1.3 MB 56.5 MB/s 
     |████████████████████████████████| 3.0 MB 77.6 MB/s 
     |████████████████████████████████| 880 kB 70.1 MB/s 
  Created wheel for sacremoses: filename=sacremoses-0.0.53-py3-none-any.whl size=895260 sha256=e560e3dbbc107c0c818069dfa01f83420d91775b61dc8a5e1153ec2f81dfe395
  Stored in directory: /root/.cache/pip/wheels/82/ab/9b/c15899bf659ba74f623ac776e861cf2eb8608c1825ddec66a4
Successfully built sacremoses


In [ ]:
import json
import copy
import string
import pandas as pd
import numpy as np

import contractions

from transformers import pipeline

from rank_bm25 import *

import nltk 
from nltk.corpus import stopwords
nltk.download('all')

from textblob import TextBlob

[nltk_data] Downloading collection 'all'
[nltk_data]    | 
[nltk_data]    | Downloading package abc to /root/nltk_data...
[nltk_data]    |   Unzipping corpora/abc.zip.
[nltk_data]    | Downloading package alpino to /root/nltk_data...
[nltk_data]    |   Unzipping corpora/alpino.zip.
[nltk_data]    | Downloading package averaged_perceptron_tagger to
[nltk_data]    |     /root/nltk_data...
[nltk_data]    |   Unzipping taggers/averaged_perceptron_tagger.zip.
[nltk_data]    | Downloading package averaged_perceptron_tagger_ru to
[nltk_data]    |     /root/nltk_data...
[nltk_data]    |   Unzipping
[nltk_data]    |       taggers/averaged_perceptron_tagger_ru.zip.
[nltk_data]    | Downloading package basque_grammars to
[nltk_data]    |     /root/nltk_data...
[nltk_data]    |   Unzipping grammars/basque_grammars.zip.
[nltk_data]    | Downloading package bcp47 to /root/nltk_data...
[nltk_data]    | Downloading package biocreative_ppi to
[nltk_data]    |     /root/nltk_data...
[nltk_data]    |   U

In [ ]:
from google.colab import drive
drive.mount('/content/drive/')

Drive already mounted at /content/drive/; to attempt to forcibly remount, call drive.mount("/content/drive/", force_remount=True).


In [ ]:
def getRestaurants(document, classifier):
    '''
    A function to extract restaurant names out of a document
    '''

    restaurants = []
    blob = TextBlob(document)
    nps = blob.noun_phrases


    if len(nps) == 0:
        return restaurants

    # collect noun phrases that contain proper nouns
    # we tried just extracting proper nouns, but this broke up multi-word restaurant names
    NNPs = []
    for word, pos in blob.tags:
        if pos == 'NNP':
            NNPs.append(word.lower())
    
    nps = []
    for np in blob.noun_phrases:
        for word in np.split(" "):
            if word in NNPs:
                nps.append(np)
                break;
    
    # return only the nps that classify as locations
    sequences = nps
    candidate_labels = ["store", "restaurant", 'office']
    hypothesis_template = "This example is the name of a {}."

    if len(sequences) > 0:
        results = classifier(sequences, candidate_labels, hypothesis_template=hypothesis_template)
        if len(sequences) > 1:
            for result in results:
                sequence = result['sequence']
                label = result['labels'][0]
                
                if label == "restaurant":
                    restaurants.append(sequence)
        else:
            sequence = results['sequence']
            label = results['labels'][0]
            
            if label == "restaurant":
                restaurants.append(sequence)
    
    return restaurants


# Part 2: Preprocess data

In [ ]:
# Load data
f = open('/content/drive/MyDrive/CS410 Final Project/reddit_data_big.json')
data = json.load(f)
f.close()

corpus = []
for entry in data:
    corpus.append(entry['body'])
    for comment in entry['comments']:
        corpus.append(comment)

# Remove stopwords, remove punctuation, and tokenize corpus
translator = str.maketrans('', '', string.punctuation + '’')
stop_words = set(stopwords.words('english'))

# removing stopwords for bm25 in tokenized_corpus, but leaving them in
# in no_contractions_corpus for comprehensible results during POS tagging in getRestaurants
no_contractions_corpus = []
tokenized_corpus = []

for doc in corpus:
    no_contractions_doc = []
    no_stop_doc = []
    for word in doc.split(" "):
        no_contractions_doc.append(contractions.fix(word))
        if word.lower() not in stop_words:
            no_stop_doc.append(contractions.fix(word).lower())
    no_contractions_corpus.append(' '.join(no_contractions_doc))
    tokenized_corpus.append(no_stop_doc)

no_punct_corpus = [doc.translate(translator) for doc in no_contractions_corpus]


In [ ]:
bm25 = BM25Okapi(tokenized_corpus)
classifier = pipeline("zero-shot-classification")

Downloading:   0%|          | 0.00/908 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/899k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/456k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.63G [00:00<?, ?B/s]

Some weights of the model checkpoint at facebook/bart-large-mnli were not used when initializing BartForSequenceClassification: ['model.encoder.version', 'model.decoder.version']
- This IS expected if you are initializing BartForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPretraining model).
- This IS NOT expected if you are initializing BartForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


# Part 3: Apply Algorithms

In [ ]:
# Queries for finding relevant documents
queries = ["mexican restaurants", "chinese restaurants", "good restaurants", "cheap restaurants"]
names = ['mex','chi','good','cheap']
N = 100

i = 0
for query in queries:
  tokenized_query = query.split(" ")
  doc_scores = bm25.get_scores(tokenized_query)

  top_N = bm25.get_top_n(tokenized_query, no_punct_corpus, n=N)
  restaurants = {}
  for doc in top_N:
      res = getRestaurants(doc, classifier)
      if len(res) == 0:
          continue
      for r in res:
          restaurants[r] = 1 if restaurants.get(r) == None else restaurants[r] + 1

  d = []
  # remove common mis-classifications
  restaurants['cu'] = 0
  restaurants['champaign'] = 0
  restaurants['champaignurbana'] = 0
  restaurants['chambana'] = 0
  for k in restaurants.keys():
    d.append([k, restaurants[k]])
  df = pd.DataFrame(d, columns=['Name', 'Count'])
  df.to_csv('/content/drive/MyDrive/CS410 Final Project/'+names[i]+'.csv')
  i += 1